In [1]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
#import data 
# data.py is part of Senteval and it is used for loading word2vec style files
import senteval
import tensorflow as tf
import logging
from collections import defaultdict
import dill
import dgm4nlp

/home/theofpa/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class EmbeddingExtractor:
    """
    This will compute a forward pass with the inference model of EmbedAlign and 
        give you the variational mean for each L1 word in the batch.
        
    Note that this takes monolingual L1 sentences only (at this point we have a traiend EmbedAlign model
        which dispenses with L2 sentences).    
        
    You don't really want to touch anything in this class.
    """

    def __init__(self, graph_file, ckpt_path, config):        
        g1 = tf.Graph()
        self.meta_graph = graph_file
        self.ckpt_path = ckpt_path
        
        self.softmax_approximation = 'botev-batch' #default
        with g1.as_default():
            self.sess = tf.Session(config=config, graph=g1)
            # load architecture computational graph
            self.new_saver = tf.train.import_meta_graph(self.meta_graph)
            # restore checkpoint
            self.new_saver.restore(self.sess, self.ckpt_path) #tf.train.latest_checkpoint(
            self.graph = g1  #tf.get_default_graph()
            # retrieve input variable
            self.x = self.graph.get_tensor_by_name("X:0")
            # retrieve training switch variable (True:trianing, False:Test)
            self.training_phase = self.graph.get_tensor_by_name("training_phase:0")
            #self.keep_prob = self.graph.get_tensor_by_name("keep_prob:0")

    def get_z_embedding_batch(self, x_batch):
        """
        :param x_batch: is np array of shape [batch_size, longest_sentence] containing the unique ids of words
        
        :returns: [batch_size, longest_sentence, z_dim]        
        """
        # Retrieve embeddings from latent variable Z
        # we can sempale several n_samples, default 1
        try:
            z_mean = self.graph.get_tensor_by_name("z:0")
            
            feed_dict = {
                self.x: x_batch,
                self.training_phase: False,
                #self.keep_prob: 1.

            }
            z_rep_values = self.sess.run(z_mean, feed_dict=feed_dict) 
        except:
            raise ValueError('tensor Z not in graph!')
        return z_rep_values

In [3]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#



# Set PATHs
# path to senteval
#PATH_TO_SENTEVAL = '../'



# import SentEval
#sys.path.insert(0, PATH_TO_SENTEVAL)

# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tks1': None}
# made dictionary a dotdict
params_senteval = dotdict(params_senteval)
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}



def prepare(params, samples):
    """
    In this example we are going to load a tensorflow model, 
    we open a dictionary with the indices of tokens and the computation graph
    """
    config = tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True
    params.extractor = EmbeddingExtractor(
        graph_file='%s.meta'%(params.ckpt_path),
        ckpt_path=params.ckpt_path,
        #config=None #run in cpu
        config=config
    )

    # load tokenizer from training
    params.tks1 = dill.load(open(params.tok_path, 'rb'))
    return

def batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    for sent in batch:
        # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
        # of the pre-trained EmbedAlign model
        # from tokens ot ids position 0 is en
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
        # sentence vector is the mean of word embeddings in context
        # [1, z_dim]
        sent_vec = np.mean(z_batch1, axis=1)
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings


# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    # define paths
    # path to senteval data
    # note senteval adds downstream into the path
    params_senteval.task_path = '/home/theofpa/Developer/SentEval/data/' 
    # path to computation graph
    # we use best model on validation AER
    # TODO: you have to point to valid paths! Use the pre-trained model linked from the top of this notebook.
    params_senteval.ckpt_path = '/home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt'
    # path to tokenizer with ids of trained Europarl data
    # out dictionary id depends on dill for pickle
    params_senteval.tok_path = '/home/theofpa/Developer/SentEval/ull-practical3-embedalign/tokenizer.pickle'
    # we use 10 fold cross validation
    params_senteval.kfold = 10
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['CR', 'MR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-21 16:42:19,474 : ***** Transfer task : CR *****




INFO:tensorflow:Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:42:21,911 : Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:42:24,014 : Generating sentence embeddings
2018-05-21 16:43:23,523 : Generated sentence embeddings
2018-05-21 16:43:23,524 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-21 16:43:26,409 : Best param found at split 1: l2reg = 8                 with score 71.39
2018-05-21 16:43:29,367 : Best param found at split 2: l2reg = 8                 with score 70.18
2018-05-21 16:43:32,319 : Best param found at split 3: l2reg = 8                 with score 70.62
2018-05-21 16:43:35,235 : Best param found at split 4: l2reg = 8                 with score 70.71
2018-05-21 16:43:38,187 : Best param found at split 5: l2reg = 4                 with score 70.47
2018-05-21 16:43:41,131 : Best param found at split 6: l2reg = 8                 with score 71.03
2018-05-21 16:43:44,061 : Best param found at split 7: l2reg = 4     

INFO:tensorflow:Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:43:54,683 : Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:43:56,584 : Generating sentence embeddings
2018-05-21 16:46:45,767 : Generated sentence embeddings
2018-05-21 16:46:45,769 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-21 16:47:03,451 : Best param found at split 1: l2reg = 8                 with score 64.38
2018-05-21 16:47:21,654 : Best param found at split 2: l2reg = 8                 with score 64.62
2018-05-21 16:47:39,013 : Best param found at split 3: l2reg = 8                 with score 64.34
2018-05-21 16:47:56,376 : Best param found at split 4: l2reg = 8                 with score 64.62
2018-05-21 16:48:14,203 : Best param found at split 5: l2reg = 8                 with score 64.64
2018-05-21 16:48:31,683 : Best param found at split 6: l2reg = 8                 with score 64.75
2018-05-21 16:48:49,187 : Best param found at split 7: l2reg = 8     

...

INFO:tensorflow:Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 17:15:09,573 : Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 17:15:11,582 : Computing embedding for train
2018-05-21 17:16:31,621 : Computed train embeddings
2018-05-21 17:16:31,621 : Computing embedding for test
2018-05-21 17:17:59,131 : Computed test embeddings
2018-05-21 17:17:59,132 : Computing embedding for dev
2018-05-21 17:18:08,133 : Computed dev embeddings
2018-05-21 17:18:08,137 : Training sklearn-LogReg with standard validation..
2018-05-21 17:18:10,647 : [('reg:0.25', 71.8), ('reg:0.5', 72.2), ('reg:1', 71.8), ('reg:2', 71.2), ('reg:4', 72.2), ('reg:8', 72.6)]
2018-05-21 17:18:10,648 : Validation : best param found is reg = 8 with score             72.6
2018-05-21 17:18:10,648 : Evaluating...
2018-05-21 17:18:11,229 : 
Dev acc : 72.6 Test acc : 74.75 for                        SICK entailment

2018-05-21 17:18:11,230 : ***** Transfer task : STS14 *****




INFO:tensorflow:Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 17:18:13,116 : Restoring parameters from /home/theofpa/Developer/SentEval/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 17:18:23,536 : deft-forum : pearson = 0.3478, spearman = 0.3630
2018-05-21 17:18:31,551 : deft-news : pearson = 0.6220, spearman = 0.5783
2018-05-21 17:18:43,334 : headlines : pearson = 0.5782, spearman = 0.5705
2018-05-21 17:18:57,320 : images : pearson = 0.6578, spearman = 0.6439
2018-05-21 17:19:10,028 : OnWN : pearson = 0.6588, spearman = 0.7165
2018-05-21 17:19:25,548 : tweet-news : pearson = 0.6233, spearman = 0.5527
2018-05-21 17:19:25,549 : ALL (weighted average) : Pearson = 0.5951,             Spearman = 0.5866
2018-05-21 17:19:25,549 : ALL (average) : Pearson = 0.5813,             Spearman = 0.5708



{'MPQA': {'devacc': 83.89, 'ntest': 10606, 'acc': 83.78, 'ndev': 10606}, 'SST2': {'devacc': 67.2, 'ntest': 1821, 'acc': 67.11, 'ndev': 872}, 'MR': {'devacc': 64.53, 'ntest': 10662, 'acc': 64.72, 'ndev': 10662}, 'STS14': {'all': {'pearson': {'wmean': 0.5951356684804012, 'mean': 0.58133806891883}, 'spearman': {'wmean': 0.5865540558636717, 'mean': 0.5708291603366306}}, 'deft-news': {'nsamples': 300, 'pearson': (0.6220449172675084, 1.608193959938536e-33), 'spearman': SpearmanrResult(correlation=0.578333454614915, pvalue=3.56018796171822e-28)}, 'deft-forum': {'nsamples': 450, 'pearson': (0.34780780187617205, 3.0532805429465504e-14), 'spearman': SpearmanrResult(correlation=0.3630115248311902, pvalue=1.833847351421204e-15)}, 'images': {'nsamples': 750, 'pearson': (0.6578237179393222, 3.6541379859852533e-94), 'spearman': SpearmanrResult(correlation=0.6438621123340441, pvalue=4.9441358581162626e-89)}, 'tweet-news': {'nsamples': 750, 'pearson': (0.6233064324372322, 6.006407366138413e-82), 'spear

In [4]:
import json
print(json.dumps(results, indent=4, sort_keys=True))

{
    "CR": {
        "acc": 70.65,
        "devacc": 70.64,
        "ndev": 3775,
        "ntest": 3775
    },
    "MPQA": {
        "acc": 83.78,
        "devacc": 83.89,
        "ndev": 10606,
        "ntest": 10606
    },
    "MR": {
        "acc": 64.72,
        "devacc": 64.53,
        "ndev": 10662,
        "ntest": 10662
    },
    "MRPC": {
        "acc": 70.96,
        "devacc": 70.64,
        "f1": 80.1,
        "ndev": 4076,
        "ntest": 1725
    },
    "SICKEntailment": {
        "acc": 74.75,
        "devacc": 72.6,
        "ndev": 500,
        "ntest": 4927
    },
    "SST2": {
        "acc": 67.11,
        "devacc": 67.2,
        "ndev": 872,
        "ntest": 1821
    },
    "STS14": {
        "OnWN": {
            "nsamples": 750,
            "pearson": [
                0.6588460662032222,
                1.5004740369354218e-94
            ],
            "spearman": [
                0.7165302641879749,
                4.038459846586195e-119
            ]
        